In [132]:
import numpy as np
from laspy.file import File
from pandas import DataFrame
from geopandas import GeoDataFrame
from shapely.geometry import Point
from IPython.display import display, clear_output
import pptk

In [7]:
file_name = "data/colonge_small/dom1l-fp_32356_5645_1_nw.xyz"

#read the file into a numpy array
lidar_points = np.loadtxt(open(file_name, "r"), delimiter=",", skiprows=0)

color = np.array(np.zeros((lidar_points.shape[0], 3)))

for i in range(0, int(lidar_points.shape[0])) :
    color[i][0] = 1

In [125]:
def compare(leader, follower, axis, color_array):
    
    cutoff = 4
    
    red = np.array([1,0,0])
    
    purple = np.array([1,0,1])
    
    edge_array = np.zeros(follower.shape)
    
    point_index = 0
    
    for point in follower:
        #search the closest y value
        index = 0
        
        closest_distance = np.Infinity
        closest_index = np.Infinity
        
        while (index < leader.shape[0]):
            distance = point[axis] - leader[index][axis]
            
            if (distance > closest_distance):
                #we are getting further away again
                break;
            else:
                closest_distance = distance
                closest_index = index
            
            index = index + 1
        
        delta = point[2] - leader[closest_index][2]
        
        if (delta > cutoff):
            color_array[point_index] = red
        elif ((-1 * delta) > cutoff):
            color_array[point_index] = purple
            
            
        point_index = point_index + 1
    
    return color_array
        

In [130]:
def threeToTwoD(threeDimData, color_array):
    
    current_pos = 0

    max_size = 0;
    
    previous_array = 0
    previous_size = 0
    
    have_to_compare = False
    
    color_index = 0
    
    #we now fit all arrays to the same size 
    while (current_pos < threeDimData.shape[0]):
        
        clear_output(wait=True)
        display("progress: " + str(current_pos / threeDimData.shape[0]))
        
        current_start = current_pos
        
        current_value = threeDimData[current_pos][0]
        
        if (current_pos != 0):
            have_to_compare = True
        
        size = 0
        
        while (current_pos < threeDimData.shape[0] and threeDimData[current_pos][0] == current_value):
            size = size + 1
            current_pos = current_pos + 1
        
        current_array = threeDimData[current_start:current_pos]
        
        if (have_to_compare): #we have to compare the current_array with the last
            color_array[current_start:current_pos] = compare(previous_array, current_array, 1, color_array[current_start:current_pos])
                
        color_index = color_index + size
        
        previous_size = size
        previous_array = current_array
        
    return color_array
    

In [ ]:
color = np.full((lidar_points.shape[0], 3), 1)
color_new = threeToTwoD(lidar_points, color)
print(color_new)

'progress: 0.6231140334899761'

In [111]:
v = pptk.viewer(lidar_points, color_new)

NameError: name 'color_new' is not defined